In [1]:
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
import gym
import numpy as np

env = gym.make('SuperMarioBros-v0', apply_api_compatibility=True, render_mode="human")


/home/edu/anaconda3/envs/tp_ap/lib/python3.9/site-packages/gym/envs/registration.py:555: UserWarning: WARN: The environment SuperMarioBros-v0 is out of date. You should consider upgrading to version `v3`.
  logger.warn(
/home/edu/anaconda3/envs/tp_ap/lib/python3.9/site-packages/gym/envs/registration.py:627: UserWarning: WARN: The environment creator metadata doesn't include `render_modes`, contains: ['render.modes', 'video.frames_per_second']
  logger.warn(


In [2]:
env.action_space

Discrete(256)

In [3]:
env = JoypadSpace(env, SIMPLE_MOVEMENT)
env.action_space

Discrete(7)

In [4]:
# size of a game frame
env.observation_space.shape

(240, 256, 3)

In [5]:
def run():
    done = True
    env.reset()
    for step in range(5000):
        action = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        if done:
            env.reset()
    
    env.close()
    

In [6]:
#state = env.reset()
#env.step(1)[4]


In [7]:
from gym.spaces import Box
from gym.wrappers import FrameStack, GrayScaleObservation, ResizeObservation

class SkipFrame(gym.Wrapper):
    def __init__(self, env, skip):
        """Return only every `skip`-th frame"""
        super().__init__(env)
        self._skip = skip

    def step(self, action):
        """Repeat action, and sum reward"""
        total_reward = 0.0
        for i in range(self._skip):
            # Accumulate reward and repeat the same action
            obs, reward, done, trunk, info = self.env.step(action)
            total_reward += reward
            if done:
                break
        return obs, total_reward, done, trunk, info

# Apply Wrappers to environment
env = SkipFrame(env, skip=4)

In [8]:
from matplotlib import pyplot as plt
#state #= env.reset()[0]
#print(state.shape)
#plt.imshow(state)

In [9]:
env = GrayScaleObservation(env)
#state = env.reset()[0]
#print(state.shape)
#plt.imshow(state)

In [10]:
env = ResizeObservation(env, shape=84)
#state = env.reset()[0]
#print(state.shape)
#plt.imshow(state)

In [11]:
env = FrameStack(env, num_stack=4)
#state = env.reset()[0]
#print(state.shape)
#plt.imshow(state[3])

In [12]:
#action = env.action_space.sample()
#print(action)
#state, reward, terminated, truncated, info = env.step(action)
#print(state[0])


In [13]:
def show_frame_stack():
    plt.figure(figsize=(10,8))
    for idx in range(state.shape[0]):
        plt.subplot(1,4,idx+1)
        plt.imshow(state[idx][:])
    plt.show()

In [14]:
#env.close()

In [15]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import mean_squared_error


class DQNAgent:
    def __init__(self, state_size, action_size):
        # we define some parameters and hyperparameters:
        # "lr" : learning rate
        # "gamma": discounted factor
        # "exploration_proba_decay": decay of the exploration probability
        # "batch_size": size of experiences we sample to train the DNN
        self.lr = 0.001
        self.gamma = 0.99
        self.exploration_proba = 1.0
        self.exploration_proba_decay = 0.005
        self.batch_size = 64
        
        # We define our memory buffer where we will store our experiences
        # We stores only the 2000 last time steps
        self.memory_buffer= list()
        self.max_memory_buffer = 2000
        
        # We creaate our model having to hidden layers of 24 units (neurones)
        # The first layer has the same size as a state size
        # The last layer has the size of actions space
        self.model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(filters=32, kernel_size=8, strides=4, activation='relu', input_shape=state_size),
        tf.keras.layers.Conv2D(filters=64, kernel_size=4, strides=2, activation='relu'),
        tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=512, activation='relu'),
        tf.keras.layers.Dense(units=action_size)
        ])     
        self.model.compile(loss="mse",
                      optimizer = Adam(learning_rate=self.lr))
        
    # The agent computes the action to perform given a state 
    def compute_action(self, current_state):
        # We sample a variable uniformly over [0,1]
        # if the variable is less than the exploration probability
        #     we choose an action randomly
        # else
        #     we forward the state through the DNN and choose the action 
        #     with the highest Q-value.
        if np.random.uniform(0,1) < self.exploration_proba:
            return env.action_space.sample()
        print("QVALUES")
        curr_state = np.array(current_state, dtype=np.float32)
        curr_state = tf.convert_to_tensor(current_state)
        # normalize
        curr_state = curr_state / 255
        q_values = self.model.predict(curr_state)[0]
        return np.argmax(q_values)
        #return env.action_space.sample()


    # when an episode is finished, we update the exploration probability using 
    # espilon greedy algorithm
    def update_exploration_probability(self):
        print("CHECK")
        self.exploration_proba = self.exploration_proba * np.exp(-self.exploration_proba_decay)
        print(self.exploration_proba)
    
    # At each time step, we store the corresponding experience
    def store_episode(self,current_state, action, reward, next_state, done):
        #We use a dictionnary to store them
        self.memory_buffer.append({
            "current_state":current_state,
            "action":action,
            "reward":reward,
            "next_state":next_state,
            "done" :done
        })
        # If the size of memory buffer exceeds its maximum, we remove the oldest experience
        if len(self.memory_buffer) > self.max_memory_buffer:
            self.memory_buffer.pop(0)
    

     #At the end of each episode, we train our model
    def train(self):
        # We shuffle the memory buffer and select a batch size of experiences
        np.random.shuffle(self.memory_buffer)
        batch_sample = self.memory_buffer[0:self.batch_size]
        
        # We iterate over the selected experiences
        for experience in batch_sample:
            # We compute the Q-values of S_t
            curr_state = experience["current_state"]
            curr_state = np.array(curr_state, dtype=np.float32)

            # LazyFrame -> np array -> torch tensor
            state_float = tf.convert_to_tensor(curr_state)
            # normalize
            state_float = state_float / 255
            q_current_state = self.model.predict(state_float)
            # We compute the Q-target using Bellman optimality equation
            q_target = experience["reward"]
            print("REWARD = ", q_target)
            if not experience["done"]:
                nxt_state = experience["next_state"]
                nxt_state = np.array(nxt_state, dtype=np.float32)
                # LazyFrame -> np array -> torch tensor
                nxt_state_float = tf.convert_to_tensor(nxt_state)
                # normalize
                nxt_state_float = nxt_state_float / 255
                q_target = q_target + self.gamma*np.max(self.model.predict(nxt_state_float)[0])
            q_current_state[0][experience["action"]] = q_target
            
            # train the model
            
            curr_state = experience["current_state"]
            curr_state = np.array(curr_state, dtype=np.float32)
            # LazyFrame -> np array -> torch tensor
            state_float = tf.convert_to_tensor(curr_state)
            # normalize
            state_float = state_float / 255
            self.model.fit(state_float, q_current_state, verbose=0)
    

2023-05-27 16:35:19.337208: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-27 16:35:19.371384: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-27 16:35:19.371907: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-27 16:35:19.951694: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
# We get the shape of a state and the actions space size
state_size = env.observation_space.shape
print(state_size)
state_size=(84,84,1)
action_size = env.action_space.n
# Number of episodes to run
n_episodes = 4000000
# Max iterations per epiode
max_iteration_ep = 500
# We define our agent
agent = DQNAgent(state_size, action_size)
batch_size = agent.batch_size
total_steps = 0

# We iterate over episodes
for e in range(n_episodes):
    print(f"EPISODE: {e}")
    # We initialize the first state and reshape it to fit 
    #  with the input layer of the DNN
    current_state = env.reset()
    current_state = np.array([current_state])
    #for step in range(max_iteration_ep):
    while True:
        total_steps = total_steps + 1
        action = agent.compute_action(current_state)
        # the envrionment runs the action and returns
        # the next state, a reward and whether the agent is done
        #next_state, reward, done, _ = env.step(action)
        #next_state = np.array([next_state])
        next_state, reward, terminated, truncated, info = env.step(action)
        done = terminated or truncated
        # We sotre each experience in the memory buffer
        agent.store_episode(current_state, action, reward, next_state, done)
        
        # if the episode is ended, we leave the loop after
        # updating the exploration probability
        if done:
            agent.update_exploration_probability()
            break
        current_state = next_state
    # if the have at least batch_size experiences in the memory buffer
    # than we tain our model
    if total_steps >= batch_size:
        print("hey")
        agent.train()

(4, 84, 84)
EPISODE: 0


2023-05-27 16:35:22.071202: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
/tmp/ipykernel_107274/3972648818.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  current_state = np.array([current_state])


CHECK
0.9950124791926823
hey
1/1 [==============================] - 0s 102ms/step
REWARD =  1.0
1/1 [==============================] - 0s 14ms/step
REWARD =  0.0
1/1 [==============================] - 0s 14ms/step
REWARD =  0.0
1/1 [==============================] - 0s 14ms/step
REWARD =  0.0
1/1 [==============================] - 0s 14ms/step
REWARD =  -1.0
1/1 [==============================] - 0s 14ms/step
REWARD =  0.0
1/1 [==============================] - 0s 14ms/step
REWARD =  -1.0
1/1 [==============================] - 0s 14ms/step
REWARD =  7.0
1/1 [==============================] - 0s 14ms/step
REWARD =  1.0
1/1 [==============================] - 0s 14ms/step
REWARD =  3.0
1/1 [==============================] - 0s 13ms/step
REWARD =  2.0
1/1 [==============================] - 0s 15ms/step
REWARD =  -1.0
1/1 [==============================] - 0s 14ms/step
REWARD =  7.0
1/1 [==============================] - 0s 14ms/step
REWARD =  -1.0
1/1 [==============================] - 0s 

In [ ]:
# Save the last model
model_path = "last_model.h5"
agent.model.save(model_path)
print("Last model saved.")

In [ ]:
env.close()